In [ ]:
import os
import cv2

In [ ]:
# Tentukan direktori data
DATA_DIR = './DATA'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Minta input dari pengguna untuk huruf kelas
class_labels = input("Masukkan huruf kelas (misalnya A B C ...): ").upper().split()

# Tentukan ukuran dataset
dataset_size = 300

cap = cv2.VideoCapture(0)

# Loop untuk setiap kelas yang dimasukkan
for label in class_labels:
    class_dir = os.path.join(DATA_DIR, label)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    print('Mengumpulkan data untuk kelas {}'.format(label))

    # Cari nomor file terakhir yang ada di direktori kelas
    existing_files = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]
    if existing_files:
        last_file_number = max([int(f.split('.')[0]) for f in existing_files])
    else:
        last_file_number = -1

    done = False
    while True:
        ret, frame = cap.read()

        # Posisi dan ukuran teks
        text = "Tekan 'Q' | Mengumpulkan data {}".format(label)
        font = cv2.FONT_HERSHEY_PLAIN
        font_scale = 1.3
        thickness = 1
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = 150
        text_y = 50
        box_coords = ((text_x, text_y - text_size[1] - 10), (text_x + text_size[0] + 10, text_y + 10))

        # Tambahkan background hitam dengan opacity 50%
        overlay = frame.copy()
        cv2.rectangle(overlay, box_coords[0], box_coords[1], (0, 0, 0), cv2.FILLED)
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)

        # Tambahkan teks putih di atas background hitam
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = last_file_number + 1
    while counter < last_file_number + 1 + dataset_size:
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(class_dir, '{}.jpg'.format(counter)), frame)

        counter += 1

cap.release()
cv2.destroyAllWindows()